### Imports

In [3]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 168 µs (started: 2023-06-06 10:30:03 -05:00)


### Parameters

In [4]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

Exp:		 exp3
Method:		 vgae
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
time: 2.33 ms (started: 2023-06-06 10:30:03 -05:00)


### Edge embeddings

In [5]:
# get edges embeddings

edge_embeddings_global(exp, method, dimension, groups_id, subgroups_id)

100%|██████████| 3/3 [47:21<00:00, 947.12s/it] 

time: 47min 21s (started: 2023-06-06 10:30:03 -05:00)


In [6]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}_{}_{}.csv".format(exp, groups_id[0], subgroups_id[groups_id[0]][0], method, dimension, "L2"), index_col=[0, 1])
df_edge_embeddings.head()

0         1         2
0 1  6.461881  0.000695  0.480581
  2  0.003822  0.842727  3.424330
  3  0.783018  0.024875  0.354545
  4  2.682043  2.122767  2.174131
  5  0.445657  1.257031  0.000274

time: 483 ms (started: 2023-06-06 11:17:26 -05:00)


### Concat edge embeddings

In [7]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    for subgroup in tqdm(subgroups_id[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}_{}_{}.csv".format(exp, group, subgroup, method, dimension, "L2"), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [subgroup] * len(df_edge_embeddings)

        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(exp, group, method, dimension, "L2"), index=True)

100%|██████████| 3/3 [02:58<00:00, 59.48s/it]

time: 2min 58s (started: 2023-06-06 11:17:27 -05:00)


In [8]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(exp, groups_id[0], method, dimension, "L2"), index_col=[0, 1])
df_edge_embeddings_concat.head()

0         1         2  subgroup
0 1  6.461881  0.000695  0.480581         1
  2  0.003822  0.842727  3.424330         1
  3  0.783018  0.024875  0.354545         1
  4  2.682043  2.122767  2.174131         1
  5  0.445657  1.257031  0.000274         1

time: 5.73 s (started: 2023-06-06 11:20:25 -05:00)


In [9]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(exp, group, method, dimension, "L2"), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}_{}.png".format(exp, group, method, dimension, "L2"))
    # plt.show()
    plt.close()

100%|██████████| 3/3 [15:08<00:00, 303.00s/it]

time: 15min 8s (started: 2023-06-06 11:20:31 -05:00)


### Outliers detection

In [10]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 2.65 ms (started: 2023-06-06 11:35:41 -05:00)


In [11]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(exp, group, method, dimension, "L2"), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [01:24<00:00, 28.16s/it]

time: 1min 24s (started: 2023-06-06 11:35:42 -05:00)


In [12]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}_{}.png".format(exp, group, method, dimension, "L2"))
    # plt.show()
    plt.close()

  0%|          | 0/3 [00:00<?, ?it/s]

Total: 12232238
Num. of inliers: 11009014
Num. of inliers: 1223224


 33%|███▎      | 1/3 [05:17<10:34, 317.28s/it]

Total: 17089033
Num. of inliers: 15380129
Num. of inliers: 1708904


###  Filter common edges

In [ ]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup))
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []

    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[str(row[-1])]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
    df_edge_embeddings_concat_filter

11009014it [04:56, 37164.20it/s]it/s]
15380129it [06:52, 37281.38it/s]9, 304.91s/it]
5439862it [02:26, 37035.81it/s]15, 375.54s/it]
100%|██████████| 3/3 [14:41<00:00, 293.95s/it]

time: 14min 41s (started: 2023-06-05 21:58:17 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
59.0049  369.1213  0.001711  4.982717e-08  0.002932         1
         641.1223  0.006827  3.968622e-07  0.011770         1
         650.2145  0.005209  4.984925e-07  0.009032         1
         977.8237  0.018792  6.723753e-06  0.033386         1
274.0123 191.0460  0.005130  3.045618e-06  0.009246         1

time: 13.2 ms (started: 2023-06-05 22:12:59 -05:00)


In [ ]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # count
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["level_0", "level_1"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]

    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:10<00:00,  3.34s/it]

time: 10 s (started: 2023-06-05 22:12:59 -05:00)


In [ ]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,745.5002,851.5293
1,200.9273,516.2880
2,139.0232,514.2847
3,676.4192,787.4748
4,139.0232,515.2878


time: 9.45 ms (started: 2023-06-05 22:13:09 -05:00)


In [ ]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

100%|██████████| 3/3 [00:48<00:00, 16.16s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
21731,59.0137,349.2758,0.796859,0.788310,0.560861,0.551982,0.807207
21738,59.0137,350.2793,0.807013,0.779306,0.576143,0.563560,0.808223
21835,59.0137,350.3033,0.854806,0.751778,0.539172,0.650983,0.883973
21666,59.0137,389.2739,0.823682,0.886260,0.723124,0.515230,0.917952
21665,59.0137,389.2907,0.887545,0.930750,0.839795,0.800743,0.914504


time: 48.7 s (started: 2023-06-05 22:13:09 -05:00)


In [ ]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
21731,59.0137,349.2758,0.796859,0.788310,0.560861,0.551982,0.807207
21738,59.0137,350.2793,0.807013,0.779306,0.576143,0.563560,0.808223
21835,59.0137,350.3033,0.854806,0.751778,0.539172,0.650983,0.883973
21666,59.0137,389.2739,0.823682,0.886260,0.723124,0.515230,0.917952
21665,59.0137,389.2907,0.887545,0.930750,0.839795,0.800743,0.914504


time: 15 ms (started: 2023-06-05 22:13:58 -05:00)


### Filter by STD and average weight

In [ ]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, dimension, groups_id, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

100%|██████████| 3/3 [00:16<00:00,  5.40s/it]


,source,target,weight
0,59.0137,349.2758,0.701044
1,59.0137,350.2793,0.706849
2,59.0137,350.3033,0.736142
3,59.0137,389.2739,0.773249
4,59.0137,389.2907,0.874667


time: 16.2 s (started: 2023-06-05 23:04:02 -05:00)


In [ ]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}_{}.csv".format(exp, groups_id[0], method, dimension, "L2"))
df_common_edges.head()

,source,target,weight
0,59.0137,349.2758,0.701044
1,59.0137,350.2793,0.706849
2,59.0137,350.3033,0.736142
3,59.0137,389.2739,0.773249
4,59.0137,389.2907,0.874667


time: 15.5 ms (started: 2023-06-05 23:04:18 -05:00)


In [ ]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}_{}.csv".format(exp, group, method, dimension, "L2"))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

  0%|          | 0/3 [00:00<?, ?it/s]

Group: WT
Num. nodes: 1901
Num. edges: 22971



 67%|██████▋   | 2/3 [00:01<00:00,  1.83it/s]

Group: zwf1^
Num. nodes: 4367
Num. edges: 521966



100%|██████████| 3/3 [00:01<00:00,  1.52it/s]

Group: pck1^
Num. nodes: 5045
Num. edges: 454531

time: 1.97 s (started: 2023-06-05 23:04:43 -05:00)
